# A simple Deep Learning Model for Classifying Sentiment

In [1]:
# Code : Import Libraries
import numpy as np 
import pandas as pd 

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import re

import warnings; warnings.simplefilter('ignore')

/home/oldmonk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Code Load Data
tweetsInfo = pd.read_csv('AllTweetInfo2.csv')
tweetsInfo.head(2)

,full_text,sentiment,retweet_count,user.friends_count,user.followers_count,user.listed_count,text_features,text_features_new,Polarity,Subjectivity
0,"Sharing my #Walk with #K9Jinky🐾 \nMy #Service,...",POS,23,9930,9925,28,sharing my walk with k9jinky my service bran...,sharing walk k9jinky service branchmanaging k9...,0.675000,0.650000
1,With this new IoT-based crash detection #servi...,NEG,1,10,6465,96,with this new iotbased crash detection service...,new iotbased crash detection service safety pr...,0.534091,0.427273


### Create Input Features and Train Validation Split

In [3]:
# Get Input Featurs
ftr_col = 'text_features'
tokenizer = Tokenizer(split=' ')
tokenizer.fit_on_texts(tweetsInfo[ftr_col].values)
X_t = tokenizer.texts_to_sequences(tweetsInfo[ftr_col].values)
X_padded = pad_sequences(X_t)

In [4]:
# Creat Train and Validation Split
y = pd.get_dummies(tweetsInfo['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X_padded,y, test_size = 0.3, random_state = 27)
#print(X_train.shape,Y_train.shape)
#print(X_test.shape,Y_test.shape)
val_size = 100
X_validate = X_test[-val_size:]
Y_validate = Y_test[-val_size:]
X_test = X_test[:-val_size]
Y_test = Y_test[:-val_size]


### A simple LSTM Network

In [5]:
def CreateModel(X_shape):
    lstm_out1, lstm_out2, l1, l2, em = 196,196,2,2,56    
    model = Sequential()
    model.add(Embedding(3000, em, input_length = X_shape))
    model.add(LSTM(lstm_out1, dropout=0.2))
    model.add(Dense(l1,activation='relu'))
    model.add(Dense(l2,activation='relu'))
   
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
    
# embed_dim = 56
# lstm_out1 = 196
# lstm_out2 = 196
# l1 = 2
# l2 = 2
# X_shape = X_padded.shape[1]

In [6]:
# Helper Functin For Model Creation
def CreatModel(batch_size, epochs, X_shape, X_train, Y_train):
    #model = KerasClassifier(build_fn = CreateModel)
    currmodel = CreateModel(X_shape)
    print(currmodel.summary())
    print()   

    print('Training Model')
    currmodel.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size, verbose = 2)
    print()
    return currmodel
  




X_shape = X_padded.shape[1]
m = CreatModel(32, 10, X_shape, X_train, Y_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 54, 56)            168000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               198352    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6         
Total params: 366,752
Trainable params: 366,752
Non-trainable params: 0
_________________________________________________________________
None

Training Model
Epoch 1/10
 - 4s - loss: 0.5294 - acc: 0.8394
Epoch 2/10
 - 2s - loss: 0.5588 - acc: 0.8394
Epoch 3/10
 - 2s - loss: 0.6471 - acc: 0.8394
Epoch 4/10
 - 2s - loss: 0.5000 - acc: 0.8394
Epoch 5/10
 - 2s - loss: 0.4412 - acc: 0.8394
Epoch 6/10
 - 2s 

In [8]:
# Evaluate on Validation Set
print("Evaluation Scores on Validation")
score,acc = m.evaluate(X_test, Y_test, verbose = 2, batch_size = 32)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))


Evaluation Scores on Validation
score: 0.96
acc: 0.81
